In [1]:
import bert_model as modeling
import re
import numpy as np
import pandas as pd
import collections
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from sklearn.cross_validation import train_test_split
from unidecode import unidecode
from tqdm import tqdm
import time

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
permulaan = [
    'bel',
    'se',
    'ter',
    'men',
    'meng',
    'mem',
    'memper',
    'di',
    'pe',
    'me',
    'ke',
    'ber',
    'pen',
    'per',
]

hujung = ['kan', 'kah', 'lah', 'tah', 'nya', 'an', 'wan', 'wati', 'ita']

def naive_stemmer(word):
    assert isinstance(word, str), 'input must be a string'
    hujung_result = re.findall(r'^(.*?)(%s)$' % ('|'.join(hujung)), word)
    word = hujung_result[0][0] if len(hujung_result) else word
    permulaan_result = re.findall(r'^(.*?)(%s)' % ('|'.join(permulaan[::-1])), word)
    permulaan_result.extend(re.findall(r'^(.*?)(%s)' % ('|'.join(permulaan)), word))
    mula = permulaan_result if len(permulaan_result) else ''
    if len(mula):
        mula = mula[1][1] if len(mula[1][1]) > len(mula[0][1]) else mula[0][1]
    return word.replace(mula, '')

In [3]:
def build_dataset(words, n_words):
    count = [['GO', 0], ['PAD', 1], ['EOS', 2], ['UNK', 3]]
    counter = collections.Counter(words).most_common(n_words)
    count.extend(counter)
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 3)
        if index == 0:
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary


def classification_textcleaning(string):
    string = re.sub(
        'http\S+|www.\S+',
        '',
        ' '.join(
            [i for i in string.split() if i.find('#') < 0 and i.find('@') < 0]
        ),
    )
    string = unidecode(string).replace('.', ' . ').replace(',', ' , ')
    string = re.sub('[^A-Za-z ]+', ' ', string)
    string = re.sub(r'[ ]+', ' ', string).strip()
    string = ' '.join(
        [i for i in re.findall('[\\w\']+|[;:\-\(\)&.,!?"]', string) if len(i)]
    )
    string = string.lower().split()
    string = [(naive_stemmer(word), word) for word in string]
    return (
        ' '.join([word[0] for word in string if len(word[0]) > 1]),
        ' '.join([word[1] for word in string if len(word[0]) > 1]),
    )

def str_idx(corpus, dic, maxlen, UNK = 3):
    X = np.zeros((len(corpus), maxlen))
    for i in range(len(corpus)):
        for no, k in enumerate(corpus[i].split()[:maxlen][::-1]):
            val = dic[k] if k in dic else UNK
            X[i, -1 - no] = val
    return X

In [4]:
import os
emotion_files = [f for f in os.listdir(os.getcwd()) if 'translated-' in f]
emotion_files

['translated-joy',
 'translated-love',
 'translated-fear',
 'translated-sadness',
 'translated-surprise',
 'translated-anger']

In [5]:
texts, labels = [], []
for f in emotion_files:
    with open(f) as fopen:
        dataset = list(filter(None, fopen.read().split('\n')))
        labels.extend([f.split('-')[1]] * len(dataset))
        texts.extend(dataset)

In [6]:
unique_labels = np.unique(labels).tolist()
labels = LabelEncoder().fit_transform(labels)
unique_labels

['anger', 'fear', 'joy', 'love', 'sadness', 'surprise']

In [7]:
for i in range(len(texts)):
    texts[i] = classification_textcleaning(texts[i])[0]

In [8]:
concat = ' '.join(texts).split()
vocabulary_size = len(list(set(concat)))
data, count, dictionary, rev_dictionary = build_dataset(concat, vocabulary_size)
print('vocab from size: %d'%(vocabulary_size))
print('Most common words', count[4:10])
print('Sample data', data[:10], [rev_dictionary[i] for i in data[:10]])

vocab from size: 14652
Most common words [('saya', 165182), ('asa', 50903), ('rasa', 50028), ('tidak', 33044), ('yang', 31373), ('untuk', 15327)]
Sample data [519, 1151, 8, 4, 103, 721, 8, 94, 114, 8] ['buah', 'parti', 'yang', 'saya', 'gi', 'natal', 'yang', 'akhir', 'malam', 'yang']


In [9]:
size_layer = 256
num_layers = 2
embedded_size = 256
dimension_output = len(unique_labels)
learning_rate = 5e-4
maxlen = 80
batch_size = 32

In [10]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

bert_config = modeling.BertConfig(
    vocab_size = len(dictionary),
    hidden_size = size_layer,
    num_hidden_layers = num_layers,
    num_attention_heads = size_layer // 4,
    intermediate_size = size_layer * 2,
)

input_ids = tf.placeholder(tf.int32, [None, maxlen], name = 'Placeholder_input_ids')
input_mask = tf.placeholder(tf.int32, [None, maxlen], name = 'Placeholder_input_mask')
segment_ids = tf.placeholder(tf.int32, [None, maxlen], name = 'Placeholder_segment_ids')
label_ids = tf.placeholder(tf.int32, [None], name = 'Placeholder_label_ids')
is_training = tf.placeholder(tf.bool, name = 'Placeholder_is_training')

In [11]:
def create_model(
    bert_config,
    is_training,
    input_ids,
    input_mask,
    segment_ids,
    labels,
    num_labels,
    use_one_hot_embeddings,
    reuse_flag = False,
):
    model = modeling.BertModel(
        config = bert_config,
        is_training = is_training,
        input_ids = input_ids,
        input_mask = input_mask,
        token_type_ids = segment_ids,
        use_one_hot_embeddings = use_one_hot_embeddings,
    )

    output_layer = model.get_pooled_output()
    hidden_size = output_layer.shape[-1].value
    with tf.variable_scope('weights', reuse = reuse_flag):
        output_weights = tf.get_variable(
            'output_weights',
            [num_labels, hidden_size],
            initializer = tf.truncated_normal_initializer(stddev = 0.02),
        )
        output_bias = tf.get_variable(
            'output_bias', [num_labels], initializer = tf.zeros_initializer()
        )

    with tf.variable_scope('loss'):
        def apply_dropout_last_layer(output_layer):
            output_layer = tf.nn.dropout(output_layer, keep_prob = 0.9)
            return output_layer

        def not_apply_dropout(output_layer):
            return output_layer

        output_layer = tf.cond(
            is_training,
            lambda: apply_dropout_last_layer(output_layer),
            lambda: not_apply_dropout(output_layer),
        )
        logits = tf.matmul(output_layer, output_weights, transpose_b = True)
        print(
            'output_layer:',
            output_layer.shape,
            ', output_weights:',
            output_weights.shape,
            ', logits:',
            logits.shape,
        )

        logits = tf.nn.bias_add(logits, output_bias, name = 'logits')
        probabilities = tf.nn.softmax(logits)
        loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
            labels = labels, logits = logits
        )
        loss = tf.reduce_mean(loss)
        correct_pred = tf.equal(tf.argmax(logits, 1, output_type = tf.int32), labels)
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

        return loss, logits, probabilities, model, accuracy

In [12]:
use_one_hot_embeddings = False
loss, logits, probabilities, model, accuracy = create_model(
    bert_config,
    is_training,
    input_ids,
    input_mask,
    segment_ids,
    label_ids,
    dimension_output,
    use_one_hot_embeddings,
)
global_step = tf.Variable(0, trainable = False, name = 'Global_Step')
optimizer = tf.contrib.layers.optimize_loss(
    loss,
    global_step = global_step,
    learning_rate = learning_rate,
    optimizer = 'Adam'
)

output_layer: (?, 256) , output_weights: (6, 256) , logits: (?, 6)


In [13]:
sess.run(tf.global_variables_initializer())

In [14]:
strings = ','.join(
    [
        n.name
        for n in tf.get_default_graph().as_graph_def().node
        if ('Variable' in n.op
        or 'Placeholder' in n.name
        or 'logits' in n.name
        or 'alphas' in n.name)
        and 'Adam' not in n.name
        and 'beta' not in n.name
        and 'OptimizeLoss' not in n.name
        and 'Global_Step' not in n.name
    ]
)

In [15]:
strings.split(',')

['Placeholder_input_ids',
 'Placeholder_input_mask',
 'Placeholder_segment_ids',
 'Placeholder_label_ids',
 'Placeholder_is_training',
 'bert/embeddings/word_embeddings',
 'bert/embeddings/token_type_embeddings',
 'bert/embeddings/position_embeddings',
 'bert/embeddings/LayerNorm/gamma',
 'bert/encoder/layer_0/attention/self/query/kernel',
 'bert/encoder/layer_0/attention/self/query/bias',
 'bert/encoder/layer_0/attention/self/key/kernel',
 'bert/encoder/layer_0/attention/self/key/bias',
 'bert/encoder/layer_0/attention/self/value/kernel',
 'bert/encoder/layer_0/attention/self/value/bias',
 'bert/encoder/layer_0/attention/output/dense/kernel',
 'bert/encoder/layer_0/attention/output/dense/bias',
 'bert/encoder/layer_0/attention/output/LayerNorm/gamma',
 'bert/encoder/layer_0/intermediate/dense/kernel',
 'bert/encoder/layer_0/intermediate/dense/bias',
 'bert/encoder/layer_0/output/dense/kernel',
 'bert/encoder/layer_0/output/dense/bias',
 'bert/encoder/layer_0/output/LayerNorm/gamma',
 

In [16]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'bert/model.ckpt')

'bert/model.ckpt'

In [17]:
vectors = str_idx(texts, dictionary, maxlen)
train_X, test_X, train_Y, test_Y = train_test_split(
    vectors, labels, test_size = 0.2
)

In [18]:
from tqdm import tqdm
import time

EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 3, 0, 0, 0

while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n' % (EPOCH))
        break

    train_acc, train_loss, test_acc, test_loss = 0, 0, 0, 0
    pbar = tqdm(
        range(0, len(train_X), batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        batch_x = train_X[i : min(i + batch_size, train_X.shape[0])]
        batch_y = train_Y[i : min(i + batch_size, train_X.shape[0])]
        np_mask = np.ones((len(batch_x), maxlen), dtype = np.int32)
        np_segment = np.ones((len(batch_x), maxlen), dtype = np.int32)
        acc, cost, _ = sess.run(
            [accuracy, loss, optimizer],
            feed_dict = {
                input_ids: batch_x,
                label_ids: batch_y,
                input_mask: np_mask,
                segment_ids: np_segment,
                is_training: True
            },
        )
        assert not np.isnan(cost)
        train_loss += cost
        train_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)
        
    pbar = tqdm(range(0, len(test_X), batch_size), desc = 'test minibatch loop')
    for i in pbar:
        batch_x = test_X[i : min(i + batch_size, test_X.shape[0])]
        batch_y = test_Y[i : min(i + batch_size, test_X.shape[0])]
        np_mask = np.ones((len(batch_x), maxlen), dtype = np.int32)
        np_segment = np.ones((len(batch_x), maxlen), dtype = np.int32)
        acc, cost = sess.run(
            [accuracy, loss],
            feed_dict = {
                input_ids: batch_x,
                label_ids: batch_y,
                input_mask: np_mask,
                segment_ids: np_segment,
                is_training: False
            },
        )
        test_loss += cost
        test_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)

    train_loss /= len(train_X) / batch_size
    train_acc /= len(train_X) / batch_size
    test_loss /= len(test_X) / batch_size
    test_acc /= len(test_X) / batch_size

    if test_acc > CURRENT_ACC:
        print(
            'epoch: %d, pass acc: %f, current acc: %f'
            % (EPOCH, CURRENT_ACC, test_acc)
        )
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1

    print('time taken:', time.time() - lasttime)
    print(
        'epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'
        % (EPOCH, train_loss, train_acc, test_loss, test_acc)
    )
    EPOCH += 1

train minibatch loop:   0%|          | 1/2463 [00:00<05:42,  7.19it/s, accuracy=0.688, cost=0.643]

epoch: 0, pass acc: 0.000000, current acc: 0.767032
time taken: 380.582350730896
epoch: 0, training loss: 0.744463, training acc: 0.728609, valid loss: 0.625143, valid acc: 0.767032



train minibatch loop:   0%|          | 1/2463 [00:00<07:07,  5.76it/s, accuracy=0.812, cost=0.46]

time taken: 396.2931480407715
epoch: 1, training loss: 0.570824, training acc: 0.791437, valid loss: 0.626640, valid acc: 0.765388



train minibatch loop:   0%|          | 1/2463 [00:00<05:33,  7.37it/s, accuracy=0.812, cost=0.438]

...

time taken: 384.2452392578125
epoch: 3, training loss: 0.461343, training acc: 0.828072, valid loss: 0.654183, valid acc: 0.760840



train minibatch loop:   0%|          | 1/2463 [00:00<05:35,  7.33it/s, accuracy=0.812, cost=0.418]

time taken: 392.4768331050873
epoch: 4, training loss: 0.423806, training acc: 0.841801, valid loss: 0.688479, valid acc: 0.752547



test minibatch loop: 100%|██████████| 616/616 [00:29<00:00, 20.83it/s, accuracy=0.783, cost=0.41] 

time taken: 396.4032061100006
epoch: 5, training loss: 0.390630, training acc: 0.853197, valid loss: 0.695388, valid acc: 0.758719

break epoch:6



In [19]:
real_Y, predict_Y = [], []

pbar = tqdm(
    range(0, len(test_X), batch_size), desc = 'validation minibatch loop'
)
for i in pbar:
    batch_x = test_X[i : min(i + batch_size, test_X.shape[0])]
    batch_y = test_Y[i : min(i + batch_size, test_X.shape[0])]
    np_mask = np.ones((len(batch_x), maxlen), dtype = np.int32)
    np_segment = np.ones((len(batch_x), maxlen), dtype = np.int32)
    predict_Y += np.argmax(
        sess.run(
            logits,
            feed_dict = {
                input_ids: batch_x,
                label_ids: batch_y,
                input_mask: np_mask,
                segment_ids: np_segment,
                is_training: False,
            },
        ),
        1,
    ).tolist()
    real_Y += batch_y.tolist()

validation minibatch loop: 100%|██████████| 616/616 [00:27<00:00, 23.75it/s]


In [20]:
from sklearn import metrics
print(metrics.classification_report(real_Y, predict_Y, target_names = unique_labels))

             precision    recall  f1-score   support

      anger       0.73      0.83      0.78      3747
       fear       0.70      0.84      0.77      3789
        joy       0.74      0.80      0.77      3929
       love       0.82      0.76      0.79      3081
    sadness       0.82      0.60      0.69      3168
   surprise       0.85      0.63      0.72      1989

avg / total       0.77      0.76      0.76     19703



In [21]:
np_mask = np.ones((1, maxlen), dtype = np.int32)
np_segment = np.ones((1, maxlen), dtype = np.int32)
text = classification_textcleaning(
    'kerajaan sebenarnya sangat bencikan rakyatnya, minyak naik dan segalanya'
)
new_vector = str_idx([text[0]], dictionary, maxlen)
sess.run(
    tf.nn.softmax(logits),
    feed_dict = {
        input_ids: new_vector,
        input_mask: np_mask,
        segment_ids: np_segment,
        is_training: False,
    },
)


array([[9.9327600e-01, 3.9941352e-03, 3.4394179e-04, 5.0133764e-04,
        1.6044716e-03, 2.8006895e-04]], dtype=float32)

In [22]:
saver.save(sess, 'bert/model.ckpt')

'bert/model.ckpt'

In [23]:
import json
with open('bert-emotion.json','w') as fopen:
    fopen.write(json.dumps({'dictionary':dictionary,'reverse_dictionary':rev_dictionary}))

In [24]:
def freeze_graph(model_dir, output_node_names):

    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            'directory: %s' % model_dir
        )

    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path

    absolute_model_dir = '/'.join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + '/frozen_model.pb'
    clear_devices = True
    with tf.Session(graph = tf.Graph()) as sess:
        saver = tf.train.import_meta_graph(
            input_checkpoint + '.meta', clear_devices = clear_devices
        )
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            tf.get_default_graph().as_graph_def(),
            output_node_names.split(','),
        )
        with tf.gfile.GFile(output_graph, 'wb') as f:
            f.write(output_graph_def.SerializeToString())
        print('%d ops in the final graph.' % len(output_graph_def.node))

In [25]:
freeze_graph('bert', strings)

INFO:tensorflow:Restoring parameters from bert/model.ckpt
INFO:tensorflow:Froze 41 variables.
INFO:tensorflow:Converted 41 variables to const ops.
375 ops in the final graph.


In [26]:
def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph

In [27]:
g = load_graph('bert/frozen_model.pb')

In [28]:
placeholder_input_ids = g.get_tensor_by_name('import/Placeholder_input_ids:0')
placeholder_input_mask = g.get_tensor_by_name('import/Placeholder_input_mask:0')
placeholder_segment_ids = g.get_tensor_by_name('import/Placeholder_segment_ids:0')
placeholder_is_training = g.get_tensor_by_name('import/Placeholder_is_training:0')
loss_logits = g.get_tensor_by_name('import/loss/logits:0')
test_sess = tf.InteractiveSession(graph = g)
test_sess.run(
    tf.nn.softmax(loss_logits),
    feed_dict = {
        placeholder_input_ids: new_vector,
        placeholder_input_mask: np_mask,
        placeholder_segment_ids: np_segment,
        placeholder_is_training: False,
    },
)

/usr/local/lib/python3.5/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


array([[9.9327600e-01, 3.9941352e-03, 3.4394179e-04, 5.0133764e-04,
        1.6044716e-03, 2.8006895e-04]], dtype=float32)